In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
data = pd.read_csv('data.csv', encoding='latin1')
data.head()

,CASE_NUMBER,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,SOC_CODE,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,WORKSITE_CITY,WORKSITE_STATE_ABB,YEAR,WORKSITE_STATE_FULL,WORKSITE
0,I-200-09285-514415,CERTIFIED,IT CATS LLC,COMPUTER SYSTEMS ANALYSTS,15-1121,SENIOR BUSINESS ANALYST,Y,109116.8,"NORTH WALES,",PA,2015,PENNSYLVANIA,"NORTH WALES, PENNSYLVANIA"
1,I-200-09341-564186,CERTIFIED-WITHDRAWN,"XPERT TECHNOLOGIES, INC.","COMPUTER OCCUPATIONS, ALL OTHER*",15-1799,QUALITY ASSURANCE ANALYST,Y,58282.0,CAMP HILL,PA,2015,PENNSYLVANIA,"CAMP HILL, PENNSYLVANIA"
2,I-200-09342-274341,CERTIFIED,"ISR INFO WAY, INC.","SOFTWARE DEVELOPERS, APPLICATIONS",15-1132,SR. SOFTWARE DEVELOPER,Y,84906.0,CHICAGO,IL,2015,ILLINOIS,"CHICAGO, ILLINOIS"
3,I-200-09364-425868,CERTIFIED,MAGIC CIRCLE FILMS INTERNATIONAL LLC,FILM AND VIDEO EDITORS,27-4032,VIDEO EDITOR,Y,39333.0,AUBURN,NY,2015,NEW YORK,"AUBURN, NEW YORK"
4,I-200-10006-413683,WITHDRAWN,SAPHIRE SOLUTIONS INC,NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,15-1142,SYSTEMS ADMINISTRATOR,Y,68638.0,BETHPAGE,NY,2015,NEW YORK,"BETHPAGE, NEW YORK"


In [4]:
print("Các giá trị duy nhất trong CASE_STATUS (trước xử lý):", data['CASE_STATUS'].unique())
print("Phân bố lớp trong CASE_STATUS (trước xử lý):", data['CASE_STATUS'].value_counts())

Các giá trị duy nhất trong CASE_STATUS (trước xử lý): ['CERTIFIED' 'CERTIFIED-WITHDRAWN' 'WITHDRAWN' 'DENIED']
Phân bố lớp trong CASE_STATUS (trước xử lý): CASE_STATUS
CERTIFIED              765099
CERTIFIED-WITHDRAWN     53049
WITHDRAWN               27297
DENIED                  11771
Name: count, dtype: int64


### Tiền xử lí dữ liệu 

In [5]:
data['CASE_STATUS'] = data['CASE_STATUS'].apply(lambda x: 1 if x == 'CERTIFIED' else 0)

features = ['PREVAILING_WAGE', 'WORKSITE', 'SOC_NAME']
X = data[features]
y = data['CASE_STATUS']


data['PREVAILING_WAGE'].fillna(data['PREVAILING_WAGE'].median())
data['WORKSITE'].fillna('Unknown')
data['SOC_NAME'].fillna('Unknown')

for col in ['WORKSITE', 'SOC_NAME']:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19788\3080726636.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19788\3080726636.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])


In [6]:
data['CASE_STATUS'].value_counts()

CASE_STATUS
1    765099
0     92117
Name: count, dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42, stratify=y)

### Huấn luyện mô hình

In [8]:
models = {
    'XGBoost': xgb.XGBClassifier(n_estimators=50, random_state=42),
    'LightGBM': lgb.LGBMClassifier(n_estimators=50, random_state=42),
    'CatBoost': cat.CatBoostClassifier(n_estimators=50, random_state=42, verbose=0)
}

results = {'Model': [], 'Accuracy': [], 'AUC': [], 'Training Time (s)': []}
predictions = {}
misclassified_data = []

for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    results['Model'].append(name)
    results['Accuracy'].append(accuracy)
    results['AUC'].append(auc)
    results['Training Time (s)'].append(training_time)
    predictions[name] = y_pred
    
    misclassified = X_test[y_test != y_pred].copy()
    misclassified['Model'] = name
    misclassified['True_Label'] = y_test[y_test != y_pred]
    misclassified['Predicted_Label'] = y_pred[y_test != y_pred]
    misclassified_data.append(misclassified)



results_df = pd.DataFrame(results)

[LightGBM] [Info] Number of positive: 512616, number of negative: 61718
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117571 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 712
[LightGBM] [Info] Number of data points in the train set: 574334, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.892540 -> initscore=2.116951
[LightGBM] [Info] Start training from score 2.116951


### Vẽ biểu đồ đường đôi 

In [9]:
fig, ax1 = plt.subplots(figsize=(8, 6))
ax1.set_xlabel('Model')
ax1.set_ylabel('Accuracy', color='tab:blue')
ax1.plot(results_df['Model'], results_df['Accuracy'], color='tab:blue', marker='o', label='Accuracy')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('AUC', color='tab:orange')
ax2.plot(results_df['Model'], results_df['AUC'], color='tab:orange', marker='s', label='AUC')
ax2.tick_params(axis='y', labelcolor='tab:orange')

fig.tight_layout()
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)
plt.title('Model Performance: Accuracy and AUC')
plt.savefig('model_performance.png')
plt.close()


### Vẽ biểu đồ phân tán các giá trị dự đoán sai

In [10]:
misclassified_df = pd.concat(misclassified_data).reset_index(drop=True)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=misclassified_df, x='PREVAILING_WAGE', y='WORKSITE', 
                hue='Model', style='True_Label', size='True_Label', 
                palette='deep', sizes=(50, 200), alpha=0.6)
plt.title('Misclassified Samples by Model')
plt.xlabel('Prevailing Wage')
plt.ylabel('Worksite(Encoded)')
plt.savefig('misclassified_samples.png')
plt.close()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19788\2068918169.py:9: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('misclassified_samples.png')



Ba mô hình XGBoost, LightGBM và CatBoost được huấn luyện trên dữ liệu H-1B Visa Petitions (2015-2019) để dự đoán chấp thuận visa, dùng các đặc trưng như lương, ngành nghề và địa điểm. Với 50 cây quyết định, kết quả trên tập kiểm tra cho thấy: 
* XGBoost (Accuracy: 0.8931, AUC: 0.6300, thời gian: 2.34s).
* LightGBM (Accuracy: 0.8930, AUC: 0.6225, thời gian: 8.24s)
* CatBoost (Accuracy: 0.8929, AUC: 0.6125, thời gian: 15.12s). 
* XGBoost cho hiệu suất tốt nhất cả về Accuracy và AUC
* LightGBM nhanh và gần bằng XGBoost, trong khi CatBoost thấp nhất.
* Biểu đồ phân tán cho thấy các mẫu dự đoán sai tập trung ở một số bang (mã hóa cao) và mức lương cao, đặc biệt với CatBoost. Cần nghiên cứu thêm để cải thiện dự đoán ở các khu vực này.
